In [1]:
import pandas as pd
from datetime import date

In [2]:
#Configuración
RUTA_BASE_INPUT = './INPUT/'
NOMBRE_CONSULTA_SA = 'CONSULTA_SQL.csv'
NOMBRE_CONSOLIDADO = 'CONSOLIDADO_DATOS_IMPORTANTES.xlsx'
RECURRENCIA = 3
NUMERO_GRUPOS = 71
# Si se deja en False, no creará todos los archivos de listas
GUARDAR_LISTAS = False
# Si se deja como False, solo guardará las listas del grupo 1 (Programación), si está activado el guardado de listas
LISTAS_TODOS_GRUPOS = False
CODIGOS_GRUPOS = ['A', 'F', 'G'] # El primer código debe ser el del grupo 1
# Mapa
COLUMNAS = {
    'LLAVE_MINTIC': 'MINTIC',
    'CODIGO_UIS': 'ESTUDIANTE',
    'NUMERO_DE_DOCUMENTO': 'DOCUMENTO',
    'APELLIDOS': 'NOMBRES',
    'NOMBRES': 'APELLIDOS',
    'EMAIL': 'EMAIL',
    'TELEFONO_MINTIC': 'TELEFONO',
    'ASIGNATURA': 'ASIGNATURA 1',
    'GRUPO1': 'GRUPO 1',
    'DOCENTE_PYTHON': 'DOCENTE',
    'HORA_INICIAL1': 'HORA INICIAL',
    'DURACION1': 'DURACION',
    # INGLES
    'ASIGNATURA2': 'ASIGNATURA 2',
    'GRUPO2': 'GRUPO 2',
    'DOCENTE_INGLES': 'DOCENTE.1',
    'HORA_INICIAL2': 'HORA INICIAL.1',
    'DURACION2': 'DURACION.1',
    # COACH
    'ASIGNATURA3': 'ASIGNATURA 3',
    'GRUPO3': 'GRUPO 3',
    'DOCENTE_COACH': 'DOCENTE.2',
    'HORA_INICIAL3': 'HORA INICIAL.2',
    'DIA': 'DIA',
    'DURACION3': 'DURACION.2'
}
COLUMNAS_ADICIONALES = {
    'pssBiblio': 'PSSW_BIBLIO',
    'hIniCad1': 'HORA_INICIAL1_',
    'hFin1': 'HORA_FINAL1',
    'hFinCad1': 'HORA_FINAL1_',
    'hIniCad2': 'HORA_INICIAL2_',
    'hFin2': 'HORA_FINAL2',
    'hFinCad2': 'HORA_FINAL2_',
    'hIniCad3': 'HORA_INICIAL3_',
    'hFin3': 'HORA_FINAL3',
    'hFinCad3': 'HORA_FINAL3_',
    'telForm': 'TELEFONO_FORMULARIO',
    'telSis': 'TELEFONO_SISTEMA_ACADEMICO',
    'nivelA': 'NIVEL_EDUCATIVO'
}
ORDEN = [
    'CODIGO_UIS',
    'NUMERO_DE_DOCUMENTO',
    'PSSW_BIBLIO',
    'NOMBRES',
    'APELLIDOS',
    'EMAIL',
    'ASIGNATURA',
    'GRUPO1',
    'HORA_INICIAL1',
    'HORA_INICIAL1_',
    'HORA_FINAL1',
    'HORA_FINAL1_',
    'DURACION1',
    'ASIGNATURA2',
    'GRUPO2',
    'HORA_INICIAL2',
    'HORA_INICIAL2_',
    'HORA_FINAL2',
    'HORA_FINAL2_',
    'DURACION2',
    'ASIGNATURA3',
    'GRUPO3',
    'DIA',
    'HORA_INICIAL3',
    'HORA_INICIAL3_',
    'HORA_FINAL3',
    'HORA_FINAL3_',
    'DURACION3',
    'LLAVE_MINTIC',
    'TELEFONO_MINTIC',
    'TELEFONO_FORMULARIO',
    'TELEFONO_SISTEMA_ACADEMICO',
    'DOCENTE_PYTHON',
    'DOCENTE_INGLES',
    'DOCENTE_COACH',
    'NIVEL_EDUCATIVO',
]
DIAS = [
    'Lunes',
    'Martes',
    'Miércoles',
    'Jueves',
    'Viernes'
]


In [3]:
# Lectura
consulta = pd.read_csv(RUTA_BASE_INPUT + NOMBRE_CONSULTA_SA, sep=';', encoding='latin-1')
consolidado = pd.read_excel(RUTA_BASE_INPUT + NOMBRE_CONSOLIDADO, engine='openpyxl')

In [4]:
# Funciones
def buscarEnDf(df, columnaKey, key, columna):
    query = df[df[columnaKey] == key][columna]
    if len(query) > 0:        
        return query.iloc[0]
    return -1
def genBibPass(documento):
    documento = str(documento)
    return documento[len(documento) - 4:]

In [5]:
# Procesamiento
listas = {}
for columna in COLUMNAS_ADICIONALES:
    listas[columna] = []
consultaE = consulta.copy()
for index, row in consultaE.iterrows():
    codigoEst = row[COLUMNAS['CODIGO_UIS']]
    listas['pssBiblio'].append(genBibPass(row[COLUMNAS['NUMERO_DE_DOCUMENTO']]))
    for i in range(1, RECURRENCIA + 1):
        indice = str(i)
        listas['hIniCad' + indice].append(str(row[COLUMNAS['HORA_INICIAL' + indice]]) + ':00:00')
        listas['hFin' + indice].append(str(row[COLUMNAS['HORA_INICIAL' + indice]] + row[COLUMNAS['DURACION' + indice]]))
        listas['hFinCad' + indice].append(str(row[COLUMNAS['HORA_INICIAL' + indice]] + row[COLUMNAS['DURACION' + indice]]) + ':00:00')
    consultaE.at[index, COLUMNAS['DIA']] = DIAS[row[COLUMNAS['DIA']] - 1]
    telForm = buscarEnDf(consolidado, 'CODIGO', codigoEst, 'TELEFONO')
    nivelA = buscarEnDf(consolidado, 'CODIGO', codigoEst, 'NIVEL_EDUCATIVO_AGRUPADO')
    listas['nivelA'].append(nivelA if nivelA != -1 else 'DESCONOCIDO')
    listas['telForm'].append(telForm if telForm != -1 else '')
    listas['telSis'].append('dummy')
for columnaNueva, lista in listas.items():
    consultaE[columnaNueva] = pd.Series(lista)

In [6]:
# Renombre
columnasAuxiliares = {v: k for k, v in COLUMNAS.items()}
columnasRenombradas = {**columnasAuxiliares, **COLUMNAS_ADICIONALES}
consultaERen = consultaE.rename(columns = columnasRenombradas)

In [7]:
consultaERenOrdenada = consultaERen[ORDEN]

In [8]:
consultaERenOrdenada.to_excel('./OUTPUT/HorariosEstudiantes_VX_' + str(date.today()) + '.xlsx', index = False)

In [13]:
nivelesEducativos = consultaERenOrdenada['NIVEL_EDUCATIVO'].unique()
estadisticasCursos = {
    'CODIGO_CURSO' : [],
    'INDICE_HOMOGENEIDAD' : [],
    'DOCENTE' : []
}
for nivel in nivelesEducativos:
    estadisticasCursos[nivel] = []
limSup = 2 if not LISTAS_TODOS_GRUPOS else RECURRENCIA + 1
for grupo in range(1, limSup):    
    for i in range(1, NUMERO_GRUPOS + 1):
        indice = str(i)
        nombreCurso = CODIGOS_GRUPOS[grupo - 1] + str(i)
        df = consultaERenOrdenada[consultaERenOrdenada['GRUPO' + str(grupo)] == nombreCurso]
        estadisticasCursos['CODIGO_CURSO'].append(nombreCurso)
        clave = 'DOCENTE'
        docenteCol = clave if grupo == 1 else clave + '.' + str(grupo - 1)
        docente = buscarEnDf(consulta, COLUMNAS['GRUPO' + str(grupo)], nombreCurso, docenteCol)
        if docente == -1:
            raise Exeption('El curso', nombreCurso, 'no tiene docente asignado.')
        estadisticasCursos['DOCENTE'].append(docente)
        maxN = -1
        totalCurso = len(df)
        for nivel in nivelesEducativos:
            cantNivel = len(df[df[COLUMNAS_ADICIONALES['nivelA']] == nivel])
            estadisticasCursos[nivel].append(cantNivel)
            if cantNivel > maxN:
                maxN = cantNivel
        estadisticasCursos['INDICE_HOMOGENEIDAD'].append(maxN / totalCurso)
        if GUARDAR_LISTAS:
            df.to_excel('./OUTPUT/CURSOS/' + nombreCurso + '.xlsx', index = False)
pd.DataFrame(estadisticasCursos).to_excel('./OUTPUT/INFORME_HOMOGENEIDAD_CURSOS_' + str(date.today()) + '.xlsx', index = False)